In [14]:
import os
from IPython.display import display, Markdown
from CodeQwenHf import CodeQwen
from ArtigenzCoder import ArtigenzCoder
import time
import json
from tqdm.auto import tqdm, trange


In [15]:
codeqwen = CodeQwen()
# artigenz_coder = ArtigenzCoder()


Loaded as API: https://qwen-codeqwen1-5-7b-chat-demo.hf.space ✔


In [18]:
def validate_config(config: dict):
    required_keys = [
    "language", "summarize_codebase", "codebase_readme_path", "files_to_summarize_paths", "codebase_summary_prompt_template", "codebase_summary_prompt_save_path", "codebase_summary_save_path", "function_description_prompt_template", "function_description_prompt_save_path", "function_description_save_path",  "function_generation_prompt_template_type1", "function_generation_prompt_type1_save_path", "function_generation_prompt_template_type2", "function_generation_prompt_type2_save_path", "function_generation_prompt_template_type3", "function_generation_prompt_type3_save_path", "chosen_function_path", "chosen_function", "original_function_save_path", "example_function_description1", "example_function_code1", "example_function_description2", "example_function_code2", "generated_function_type1_save_dir", "generated_function_type2_save_dir", "generated_function_type3_save_dir", "run_codebleu", "codebleu_type1_save_dir", "codebleu_type2_save_dir", "codebleu_type3_save_dir"     
    ]
    
    for key in required_keys:
        if key not in config:
            raise ValueError(f"Missing required key: {key}")


In [19]:
remaining_confs_file = "localllm_remaining_confs.txt"
with open(remaining_confs_file, 'r') as f:
    remaining_confs_paths = f.read().splitlines()

remaining_confs = []
for cf_path in remaining_confs_paths:
    conf = json.load(open(cf_path))
    try:
        validate_config(conf)
        remaining_confs.append(conf)
    except Exception as e:
        print(f"Error in {cf_path}: {e}")


In [20]:
len(remaining_confs)


10

In [21]:
def load_func_generation_prompts(config):
    with open(config['function_generation_prompt_type1_save_path'], 'r') as f:
        function_generation_prompt_type1 = f.read()
    
    with open(config['function_generation_prompt_type2_save_path'], 'r') as f:
        function_generation_prompt_type2 = f.read()

    function_generation_prompt_type3 = ""
    if config["function_generation_prompt_type3_save_path"] != "":
        with open(config['function_generation_prompt_type3_save_path'], 'r') as f:
            function_generation_prompt_type3 = f.read()

    return function_generation_prompt_type1, function_generation_prompt_type2, function_generation_prompt_type3


In [22]:
model_dict = {
    # "GPT-3_5-Turbo": OpenAIModel("gpt-3.5-turbo"), 
    # "GPT-4": OpenAIModel("gpt-4"), 
    # "DeepSeek-Coder-V2": DeepSeek(), 
    "CodeQwen1_5-7B-Chat": codeqwen, 
    # "Artigenz-Coder-DS-6_7B": artigenz_coder
    }


In [23]:
for conf in tqdm(remaining_confs, desc="Remaining configurations"):
    print("-----------------------------------")
    function_generation_prompt_type1, function_generation_prompt_type2, function_generation_prompt_type3 = load_func_generation_prompts(conf)
    function_generation_prompts = {
        "type1": function_generation_prompt_type1,
        "type2": function_generation_prompt_type2,
        "type3": function_generation_prompt_type3
    }

    generated_function_save_dirs = {
        "type1": conf['generated_function_type1_save_dir'],
        "type2": conf['generated_function_type2_save_dir'],
        "type3": conf['generated_function_type3_save_dir']
    }

    model_names = ["GPT-3_5-Turbo", "GPT-4", "DeepSeek-Coder-V2", "CodeQwen1_5-7B-Chat", "Artigenz-Coder-DS-6_7B"]
    prompt_types = ["type1", "type2", "type3"]

    generated_function_save_paths = {t: {m: [] for m in model_names} for t in prompt_types}
    generation_count = 1


    for prompt_type in ["type1", "type2", "type3"]:
    # for prompt_type in ["type3"]:
        if prompt_type == "type3" and conf["generated_function_type3_save_dir"] == "":
            print("Skipping function generation prompt type 3")
            continue
        
        print(f"Running prompt type {prompt_type}")

        for model_name, model in model_dict.items():
            print(f"Running {model_name} model")

            for i in range(1, generation_count + 1):

                generated_function = model.generate(function_generation_prompts[prompt_type])

                filename = conf["chosen_function"].split(".")
                generated_function_save_filename = f"{filename[0]}_{i}.{filename[1]}"
                generated_function_save_path = f"{generated_function_save_dirs[prompt_type]}/{model_name}/GENERATED-{generated_function_save_filename}"
                model.write_to_file(generated_function_save_path)
                
                generated_function_save_paths[prompt_type][model_name].append(generated_function_save_path)
                print(f"Generated function {i}")



Remaining configurations:   0%|          | 0/10 [00:00<?, ?it/s]

-----------------------------------
Running prompt type type1
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type2
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type3
Running CodeQwen1_5-7B-Chat model
Generated function 1
-----------------------------------
Running prompt type type1
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type2
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type3
Running CodeQwen1_5-7B-Chat model
Generated function 1
-----------------------------------
Running prompt type type1
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type2
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type3
Running CodeQwen1_5-7B-Chat model
Generated function 1
-----------------------------------
Running prompt type type1
Running CodeQwen1_5-7B-Chat model
Generated function 1
Running prompt type type2
Running CodeQwen1_5-